In [0]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import multiprocessing
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed
import time

In [0]:
row_count = 5000
hotel_ids = [i for i in range(1,11)]
col_count = 21
X_vars = [np.random.rand(row_count, 1) * np.random.randint(2, 10) for i in range(col_count)]
Y_vars = [x_var * (index + 2) for index, x_var in enumerate(X_vars)]

X_cols = [f"X{i}" for i in range(len(Y_vars))]
Y_cols = [f"Y{i}" for i in range(len(Y_vars))]
hotel_df = pd.DataFrame(hotel_ids, columns=['HotelID'])
all_data = pd.DataFrame(
    np.concatenate(Y_vars + X_vars, axis=1), columns=Y_cols + X_cols
)

hotel_df = pd.merge(hotel_df, all_data, how='cross')

In [0]:
hotel_df_sprk = spark.createDataFrame(hotel_df)

In [0]:
def train_inner_model(train_data: pd.DataFrame, y_col: str) -> None:
    model_ = lgb.LGBMRegressor(verbose=-1, n_jobs=1)
    model_.fit(X=train_data[X_cols], y=train_data[y_col]) 

    return y_col,model_

In [0]:
# Model train with ThreadPoolExecutor
def train_model(id, train_data: pd.DataFrame):
    model_list_tp_ = {}
    n_threads = 11
    Y_cols = ['Y0']  # Assuming Y_cols is predefined
    # X_cols = ['X0', 'X1']  # Replace with the actual feature columns from your data

    # Train models for each target (Y_col) in parallel
    with ThreadPoolExecutor(max_workers=n_threads) as executor:
        future_to_target = {executor.submit(
            train_inner_model, train_data, y_col): y_col for y_col in Y_cols}
        
        for future in as_completed(future_to_target):
            forecast_point = future_to_target[future]                    
            try:
                col_, model = future.result()
            except Exception as exc:
                print(exc)
            else:
                model_list_tp_[col_] = model

    # Make predictions using the trained model
    pred_list = model_list_tp_['Y0'].predict(train_data[X_cols].reset_index(drop=True))

    # Convert predictions to native Python float
    pred_list = [float(num) for num in pred_list]
    
    # Create a Pandas DataFrame with the HotelID and predictions
    result_df = pd.DataFrame({
        'HotelID': train_data['HotelID'],
        'Y0': pred_list
    })
    
    return result_df

# Apply the train_model function using applyInPandas
output = hotel_df_sprk.groupby('HotelID').applyInPandas(
    train_model, schema="HotelID integer, Y0 double"
)

display(output)

HotelID,Y0
7,0.27841063974894414
7,4.691943375631299
7,0.2838056093589325
7,5.490857491225539
7,0.2103166135175545
7,3.4244966164507313
7,0.8699813832999809
7,4.876195430901708
7,1.2934002162479719
7,3.9419022741533483


In [0]:
def train_inner_model2(train_data: pd.DataFrame, y_col: str) -> None:
    model_ = lgb.LGBMRegressor(verbose=-1, num_threads=1)
    model_.fit(X=train_data[X_cols], y=train_data[y_col]) 

    return y_col,model_

# Function to train the model on each worker node
def train_model2(id, train_data: pd.DataFrame):
    model_list_pp_ = {}
    n_threads = 4
    Y_cols = ['Y0']  # Assuming Y_cols is predefined

    # Train models for each target (Y_col) in parallel
    with ProcessPoolExecutor(max_workers=n_threads) as executor:
        future_to_target = {executor.submit(
            train_inner_model2, train_data, y_col): y_col for y_col in Y_cols}
        
        for future in as_completed(future_to_target):
            forecast_point = future_to_target[future]                    
            try:
                col_, model = future.result()
                print(f"Trained model for: {col_}")
            except Exception as exc:
                print(f"Model training failed for {forecast_point}: {exc}")
            else:
                model_list_pp_[col_] = model

    # Debugging: Check if 'Y0' model was successfully trained
    if 'Y0' not in model_list_pp_:
        raise KeyError("'Y0' not found in model_list_pp_")

    # Make predictions using the trained model
    pred_list = model_list_pp_['Y0'].predict(train_data[X_cols].reset_index(drop=True))

    # Convert predictions to native Python float
    pred_list = [float(num) for num in pred_list]
    
    # Create a Pandas DataFrame with the HotelID and predictions
    result_df = pd.DataFrame({
        'HotelID': train_data['HotelID'],
        'Y0': pred_list
    })
    
    return result_df

# Apply the train_model function using applyInPandas
output = hotel_df_sprk.groupby('HotelID').applyInPandas(
    train_model2, schema="HotelID integer, Y0 double"
)

display(output)


org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 6825.0 failed 4 times, most recent failure: Lost task 0.3 in stage 6825.0 (TID 40225) (10.139.64.13 executor 11): org.apache.spark.api.python.PythonException: 'KeyError: "'Y0' not found in model_list_pp_"', from , line 31. Full traceback below:
Traceback (most recent call last):
 File " ", line 31, in train_model2
KeyError: "'Y0' not found in model_list_pp_"

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:694)
	at org.apache.spark.sql.execution.python.PythonArrowOutput$$anon$1.read(PythonArrowOutput.scala:110)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:647)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.execution.collect.UnsafeRowBatchUtils$.encodeUnsafeRows(UnsafeRowBatchUtils.scala:80)
	at org.apache.spark.sql.execution.collect.Collector.$anonfun$processFunc$1(Collector.scala:186)
	at org.apache.spark.scheduler.ResultTask.$anonfun$runTask$3(ResultTask.scala:75)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ResultTask.$anonfun$runTask$1(ResultTask.scala:75)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:55)
	at org.apache.spark.scheduler.Task.doRunTask(Task.scala:174)
	at org.apache.spark.scheduler.Task.$anonfun$run$4(Task.scala:137)
	at com.databricks.unity.EmptyHandle$.runWithAndClose(UCSHandle.scala:126)
	at org.apache.spark.scheduler.Task.$anonfun$run$1(Task.scala:137)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.Task.run(Task.scala:96)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$13(Executor.scala:902)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1697)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$4(Executor.scala:905)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:760)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:750)

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:3420)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:3342)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:3331)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:3331)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGScheduler.scala:1439)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1$adapted(DAGScheduler.scala:1439)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.scheduler.DAGScheduler.handleTaskSetFailed(DAGScheduler.scala:1439)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:3631)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:3569)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:3557)
	at org.apache.spark.util.EventLoop$$anon$1.run(E